# I. Distinguer les commentaires gentils et méchants

In [1]:
!wget http://theo.delemazure.fr/perso/comments.csv
!wget http://theo.delemazure.fr/perso/labels.csv

--2020-04-25 09:31:01--  http://theo.delemazure.fr/perso/comments.csv
Resolving theo.delemazure.fr (theo.delemazure.fr)... 217.160.0.80, 2001:8d8:100f:f000::267
Connecting to theo.delemazure.fr (theo.delemazure.fr)|217.160.0.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61471899 (59M) [text/csv]
Saving to: ‘comments.csv’

comments.csv        100%[===================>]  58.62M  45.9MB/s    in 1.3s    

2020-04-25 09:31:03 (45.9 MB/s) - ‘comments.csv’ saved [61471899/61471899]

--2020-04-25 09:31:04--  http://theo.delemazure.fr/perso/labels.csv
Resolving theo.delemazure.fr (theo.delemazure.fr)... 217.160.0.80, 2001:8d8:100f:f000::267
Connecting to theo.delemazure.fr (theo.delemazure.fr)|217.160.0.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3080081 (2.9M) [text/csv]
Saving to: ‘labels.csv’

labels.csv          100%[===================>]   2.94M  --.-KB/s    in 0.1s    

2020-04-25 09:31:04 (19.8 MB/s) - ‘labels.csv’ saved [3080

In [0]:
import pandas as pd
import re
import numpy as np
import torch 
import torch as th
import torch.autograd as ag
import torch.nn.functional as F
import torch.nn as nn


In [0]:
COMMENT = 'comment_text'
LIMIT = 20000
comments = pd.read_csv('comments.csv')[COMMENT].to_numpy()[:LIMIT]
labels = pd.read_csv('labels.csv').to_numpy()[:LIMIT,1:]

In [0]:
def create_vocab(comments):
  comments_split = [com.split() for com in comments]
  vocab = dict()
  pos = 0
  txtidx = []
  maxlength = 0

  for comment in comments_split:
      isent = []
      maxlength = max(maxlength,len(comment))
      for w in comment:
          if w not in vocab:
              vocab[w] = pos
              pos +=1
          isent.append(vocab[w])
      txtidx.append(torch.LongTensor(list(set(isent))))
  print("Number of words : ",len(vocab))
  print("Maximum length of comment : ",maxlength)
  print("Number of comments :",len(comments_split))
  print(txtidx[0])
  vocab_size = len(vocab)

  return txtidx,vocab_size
            

In [5]:
txtidx,vocab_size = create_vocab(comments)
labl = torch.from_numpy(labels).float().cuda()

Number of words :  65553
Maximum length of comment :  1403
Number of comments : 20000
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40])


In [0]:
class CBOW_classifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim,out_dim):
        super(CBOW_classifier, self).__init__()
        self.embeddings = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim)
        self.lin = nn.Linear(embedding_dim,out_dim)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        out = self.embeddings(inputs)
        out = out.sum(dim=0)
        return self.activation(self.lin(out))
        #return self.lin(out)

On essaie tout d'abord de classer les commentaires "méchant" et les commentaire "gentil" sans distinction de méchanceté, avec un simple classifieur.


In [0]:

classifier = CBOW_classifier(vocab_size=vocab_size,embedding_dim=20,out_dim=1)
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.05)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        loss = loss_function(probs[0],torch.max(labl[index]))
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if pred.item() == max(labl[index]).item():
          acc += 1
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

0 2000 / 10000  :  0.4343559719312781
0 4000 / 10000  :  0.3632017318294417
0 6000 / 10000  :  0.33406375427805673
0 8000 / 10000  :  0.32792279576939304
0 10000 / 10000  :  0.3175518235417292
0 tensor(0.3175) 0.9
1 2000 / 10000  :  0.2424696845493059
1 4000 / 10000  :  0.21792053848423043
1 6000 / 10000  :  0.20899524106882397
1 8000 / 10000  :  0.20420274091878673
1 10000 / 10000  :  0.20011937717209222
1 tensor(0.2001) 0.927
2 2000 / 10000  :  0.1463155741689204
2 4000 / 10000  :  0.14654869161626344
2 6000 / 10000  :  0.14349548370366416
2 8000 / 10000  :  0.14318144138134453
2 10000 / 10000  :  0.14533826429517951
2 tensor(0.1453) 0.946
3 2000 / 10000  :  0.10335751829772308
3 4000 / 10000  :  0.10961448553056233
3 6000 / 10000  :  0.10908363063765836
3 8000 / 10000  :  0.10762389610224715
3 10000 / 10000  :  0.10828446125862587
3 tensor(0.1083) 0.961
4 2000 / 10000  :  0.08698426514282531
4 4000 / 10000  :  0.08157656937427717
4 6000 / 10000  :  0.08177402532583555
4 8000 / 10000

On obtien une accuracy de train de 99% ! Qu'en est il de la validation ?

In [0]:
def test_model(classifier,txtidx,lbl):
  classifier.eval()
  
  M = np.zeros((2,2))
  for i in range(10000):
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(max(labl[i]))
    M[real,pred] += 1
  print("train : ", M)

  M = np.zeros((2,2))
  for i in range(10000,20000):
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(max(labl[i]))
    M[real,pred] += 1
  print("val : ", M)



train :  [[8951.   19.]
 [  31.  999.]]
val :  [[8673.  296.]
 [ 428.  603.]]


pas ouf, quasiment 50% d'erreur pour les commentaires méchants... on voudrait plutot l'inverse en réalité. Peut être que cela est du au fort taux de commentaire gentil dans le dataset ? Il y a en effet 90% de commentaire gentil, l'entrainement est biaisé. Voyons si l'ont peut faire mieux avec une distribution 50/50

In [0]:
c = 0
for i in range(20000):
  if sum(labl[i]) == 0:
    c += 1
print(c/200)

89.695


# II. Augmenter le nombre de commentaire méchants


In [7]:
!wget http://theo.delemazure.fr/perso/good_comments.csv
!wget http://theo.delemazure.fr/perso/bad_comments.csv
!wget http://theo.delemazure.fr/perso/bad_labels.csv

--2020-04-25 09:31:46--  http://theo.delemazure.fr/perso/good_comments.csv
Resolving theo.delemazure.fr (theo.delemazure.fr)... 217.160.0.80, 2001:8d8:100f:f000::267
Connecting to theo.delemazure.fr (theo.delemazure.fr)|217.160.0.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56659184 (54M) [text/csv]
Saving to: ‘good_comments.csv’

good_comments.csv   100%[===================>]  54.03M  40.0MB/s    in 1.4s    

2020-04-25 09:31:47 (40.0 MB/s) - ‘good_comments.csv’ saved [56659184/56659184]

--2020-04-25 09:31:49--  http://theo.delemazure.fr/perso/bad_comments.csv
Resolving theo.delemazure.fr (theo.delemazure.fr)... 217.160.0.80, 2001:8d8:100f:f000::267
Connecting to theo.delemazure.fr (theo.delemazure.fr)|217.160.0.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4785381 (4.6M) [text/csv]
Saving to: ‘bad_comments.csv’

bad_comments.csv    100%[===================>]   4.56M  26.5MB/s    in 0.2s    

2020-04-25 09:31:49 (26.5 MB/s)

In [0]:
LIMIT = 10000
good_comments = pd.read_csv('good_comments.csv')[COMMENT].to_numpy()[:LIMIT]
bad_comments  = pd.read_csv('bad_comments.csv')[COMMENT].to_numpy()[:LIMIT]

comments_split = [com.split() for com in good_comments] + [com.split() for com in bad_comments]
labels = np.array([0]*10000 + [1]*10000)

randomidx = np.arange(LIMIT*2)
np.random.shuffle(randomidx)
comments_split = [comments_split[i] for i in randomidx]
labels = [labels[i] for i in randomidx]

In [66]:
vocab = dict()
pos = 0
txtidx = []
maxlength = 0

for comment in comments_split:
    isent = []
    maxlength = max(maxlength,len(comment))
    for w in comment:
        if w not in vocab:
            vocab[w] = pos
            pos +=1
        isent.append(vocab[w])
    txtidx.append(torch.LongTensor(list(set(isent))))

print("Number of words : ",len(vocab))
print("Maximum length of comment : ",maxlength)
print("Number of comments :",len(comments_split))
print(txtidx[0])

vocab_size = len(vocab)
labl = torch.tensor(labels).float().cuda()
print(labl.shape)

Number of words :  59255
Maximum length of comment :  1403
Number of comments : 20000
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25])
torch.Size([20000])


In [0]:
classifier = CBOW_classifier(vocab_size=vocab_size,embedding_dim=20,out_dim=1)
#loss_function = nn.MultiLabelSoftMarginLoss()
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.05)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        #loss = loss_function(probs[None,:], labl[index][None,:])
        loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if pred.item() == labl[index].item():
          acc += 1
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

0 2000 / 10000  :  0.6571886407070723
0 4000 / 10000  :  0.5909046841788572


KeyboardInterrupt: ignored

In [0]:
classifier.eval()

M = np.zeros((2,2))
for i in range(10000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i])
  M[real,pred] += 1
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i])
  M[real,pred] += 1
print("val : ", M)


ça marche déjà Vachement mieux ! 

# III. Meilleur classifieur

In [0]:

class MLP_classifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim,out_dim, hidden_dim = 10):
        super(MLP_classifier, self).__init__()
        self.embeddings = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim)
        self.couches = []
        self.hidden = nn.Linear(embedding_dim,hidden_dim)
        
        self.fin = nn.Linear(hidden_dim,out_dim)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        out = self.embeddings(inputs)
        out = out.sum(dim=0)
        out = F.relu(self.hidden(out))

        return self.activation(self.fin(out))
        #return self.lin(out)

In [0]:
classifier = MLP_classifier(vocab_size=vocab_size,embedding_dim=25,out_dim=1,hidden_dim=10)
#loss_function = nn.MultiLabelSoftMarginLoss()
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.05)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        #loss = loss_function(probs[None,:], labl[index][None,:])
        loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if pred.item() == labl[index].item():
          acc += 1
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

0 2000 / 10000  :  0.5763937877141696
0 4000 / 10000  :  0.5367806184944673
0 6000 / 10000  :  0.5043264860549675
0 8000 / 10000  :  0.47788728107615014
0 10000 / 10000  :  0.46585640009313434
0 tensor(0.4658) 0.778
1 2000 / 10000  :  0.308221066099456
1 4000 / 10000  :  0.29060115370639533
1 6000 / 10000  :  0.2888434083089369
1 8000 / 10000  :  0.28218026154636516
1 10000 / 10000  :  0.28086863862167466
1 tensor(0.2809) 0.882
2 2000 / 10000  :  0.20263487151290488
2 4000 / 10000  :  0.19914002548488685
2 6000 / 10000  :  0.1985384201403359
2 8000 / 10000  :  0.1949171411318993
2 10000 / 10000  :  0.19141309882941418
2 tensor(0.1914) 0.923
3 2000 / 10000  :  0.1288643284044366
3 4000 / 10000  :  0.132069470614724
3 6000 / 10000  :  0.12888761082576264
3 8000 / 10000  :  0.13145040652054163
3 10000 / 10000  :  0.13177742334780354
3 tensor(0.1318) 0.951
4 2000 / 10000  :  0.08138968623716154
4 4000 / 10000  :  0.08728605796468648
4 6000 / 10000  :  0.08739744994934051
4 8000 / 10000  : 

In [0]:
classifier.eval()

M = np.zeros((2,2))
for i in range(10000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i])
  M[real,pred] += 1
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i])
  M[real,pred] += 1
print("val : ", M)


train :  [[5011.   16.]
 [  14. 4959.]]
val :  [[4229.  744.]
 [ 803. 4224.]]


# IV. Prédire la cause de méchanceté


In [45]:
bad_labels = pd.read_csv('bad_labels.csv').to_numpy()[:LIMIT,1:]
labels = np.concatenate([np.zeros((LIMIT,6)),bad_labels],0)
comments_split = [com.split() for com in good_comments] + [com.split() for com in bad_comments]

randomidx = np.arange(LIMIT*2)
np.random.shuffle(randomidx)
comments_split = [comments_split[i] for i in randomidx]
labels = [labels[i] for i in randomidx]

vocab = dict()
pos = 0
txtidx = []
maxlength = 0

for comment in comments_split:
    isent = []
    maxlength = max(maxlength,len(comment))
    for w in comment:
        if w not in vocab:
            vocab[w] = pos
            pos +=1
        isent.append(vocab[w])
    txtidx.append(torch.LongTensor(list(set(isent))))

print("Number of words : ",len(vocab))
print("Maximum length of comment : ",maxlength)
print("Number of comments :",len(comments_split))
print(txtidx[0])

vocab_size = len(vocab)
labl = torch.tensor(labels).float().cuda()
print(labl.shape)

Number of words :  59255
Maximum length of comment :  1403
Number of comments : 20000
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
torch.Size([20000, 6])


In [0]:
classifier = CBOW_classifier(vocab_size=vocab_size,embedding_dim=25,out_dim=6)
loss_function = nn.MultiLabelSoftMarginLoss()
#loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.01)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        loss = loss_function(probs[None,:], labl[index][None,:].cuda())
        #loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx))

0 2000 / 10000  :  0.7270653148840045
0 4000 / 10000  :  0.7167950434874344
0 6000 / 10000  :  0.7119213879396566
0 8000 / 10000  :  0.7090616551678335
0 10000 / 10000  :  0.7066899072719772
0 tensor(0.7067)
1 2000 / 10000  :  0.6977448074623249
1 4000 / 10000  :  0.6963651923723119


KeyboardInterrupt: ignored

In [0]:
classifier.eval()

j = 0
M = np.zeros((2,2))
for i in range(10000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs[j] > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i,j])
  M[real,pred] += 1
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs[j] > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i,j])
  M[real,pred] += 1
print("val : ", M)


train :  [[9.585e+03 0.000e+00]
 [4.130e+02 2.000e+00]]
val :  [[9572.   10.]
 [ 418.    0.]]


## V. Utiliser les embeddings FastText

In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

--2020-04-24 09:25:05--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  5.45MB/s    in 3m 2s   

2020-04-24 09:28:07 (3.58 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [0]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')
weights = torch.FloatTensor(model.vectors) 

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


NameError: ignored

In [0]:
embedding = nn.Embedding.from_pretrained(weights)

AttributeError: ignored

In [0]:
import gensim.downloader as api

word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
LIMIT = 10000
good_comments = pd.read_csv('good_comments.csv')[COMMENT].to_numpy()[:LIMIT]
bad_comments  = pd.read_csv('bad_comments.csv')[COMMENT].to_numpy()[:LIMIT]

comments_split = [com.split() for com in good_comments] + [com.split() for com in bad_comments]
labels = np.array([0]*10000 + [1]*10000)

randomidx = np.arange(LIMIT*2)
np.random.shuffle(randomidx)
comments_split = [comments_split[i] for i in randomidx]
labels = [labels[i] for i in randomidx]
labl = torch.tensor(labels).float().cuda()

In [0]:
print(comments_split[15])
print(word_vectors[comments_split[15]])

['get', 'a', 'proper', 'job', 'daft', 'cunt']
[[ 1.4433e-01  4.3951e-01  5.8324e-01 -7.4477e-01 -4.9797e-01  8.6928e-02
  -2.9798e-01  3.9964e-01  1.4083e-01 -3.6578e-01  3.3322e-01  4.7181e-01
   1.9960e-01  1.8278e-01  1.7176e-01 -3.4297e-01  4.2520e-02  3.9309e-01
  -6.6272e-01  6.2738e-01  2.2845e-01  4.4657e-01  1.1174e-01 -4.1396e-01
   7.9795e-02  7.8823e-02 -2.6135e-01 -8.0966e-01  5.2807e-01 -4.7327e-01
   5.5901e-02  8.5753e-01  1.9722e-01  3.0782e-01  3.9039e-01  3.1804e-01
  -5.3604e-01  1.8350e-01  8.1918e-02  2.7753e-01 -2.5792e-01 -3.7158e-01
  -2.2150e-01 -1.0916e+00 -5.2179e-01  1.2665e-01 -4.8591e-01 -2.6716e-01
   3.7119e-01 -1.0525e+00 -3.0202e-01 -1.6279e-01 -3.2528e-01  9.5493e-01
   1.4915e-01 -2.4934e+00  2.7505e-01  1.6308e-01  1.8972e+00  2.1172e-01
   1.0776e-01  1.1318e+00 -8.5962e-01  6.0199e-02  8.3713e-01  3.0006e-02
   5.9344e-01  4.2821e-01 -1.3722e-01 -4.9128e-01  4.3155e-03 -5.1829e-01
  -5.9878e-02 -5.8476e-01  6.2621e-02  3.1269e-01 -5.2613e-01 -3.5

In [0]:
class MLP_classifier_2(nn.Module):

    def __init__(self,embedding_dim,hidden_dim,out_dim):
        super(MLP_classifier_2, self).__init__()
        self.hidden = nn.Linear(embedding_dim,hidden_dim)
        self.lin = nn.Linear(hidden_dim,out_dim)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        out = inputs.sum(dim=0)
        out = F.relu(self.hidden(out))
        return self.activation(self.lin(out))
        #return self.lin(out)

In [0]:
classifier = MLP_classifier_2(100,hidden_dim=20,out_dim=1)
#loss_function = nn.MultiLabelSoftMarginLoss()
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.01)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = comments_split[index]
        print(x)
        probs = classifier(th.Tensor(word_vectors[x]).cuda()).cuda()
        #loss = loss_function(probs[None,:], labl[index][None,:])
        loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if pred.item() == labl[index].item():
          acc += 1
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

['k', 'what', 'the', 'fuck', 'man', 'why', 'did', 'you', 'add', 'a', 'new', 'source', 'the', 'source', 'given', 'was', 'invalidmeaning', 'that', 'genre', "can't", 'be', 'listed', "anywaydon't", 'see', 'why', 'that', 'was', 'cleared', 'as', 'a', 'source', 'from', 'the', 'getgo', 'so', 'in', 'that', 'casehow', 'how', 'does', 'that', 'differ', 'from', 'me', 'adding', 'a', 'deathcore', 'genre', 'just', 'get', 'rid', 'of', 'the', 'fucking', 'melodic', 'death', 'metal', 'seciton', 'of', 'it', "it's", 'pretty', 'muchfucked', 'up']


KeyError: ignored

# VI. Conv1D 

In [37]:
h1 = 4 # dimension of embeddings, the input size for convolution
h2 = 2 # output dimension (filter size) for the convolution
embLayer = th.nn.Embedding(num_embeddings=vocab_size, embedding_dim=h1)
print(txtidx[2])
embs = embLayer(txtidx[2])
print(len(txtidx[2]))
print(embs.shape)

conv1 = th.nn.Conv1d(in_channels=4,out_channels=2,kernel_size=3,padding=1)
tmp=embs.view(1,4,-1)
res = conv1(tmp)
print("embs : ",embs.shape)
print("tmp  : ",tmp.shape)
print("conv : ",res.shape)

m = th.nn.MaxPool1d(res.shape[-1])
res = m(res)
res = res.view(h2)
print("mp : ",res.shape)

tensor([ 0, 12, 16, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
        40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
        58, 59, 60, 61, 62, 63, 64])
43
torch.Size([43, 4])
embs :  torch.Size([43, 4])
tmp  :  torch.Size([1, 4, 43])
conv :  torch.Size([1, 2, 43])
mp :  torch.Size([2])


In [0]:
class Conv1D_classifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim,out_dim, feat_size=10, kernel_size=3):
        super(Conv1D_classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim)
        
        self.conv = nn.Conv1d(embedding_dim,feat_size,kernel_size,padding=1)
        
        self.lin = nn.Linear(feat_size,out_dim)      
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        embs = self.embeddings(inputs)
        tmp=embs.view(1,self.embedding_dim,-1)
        tmp = self.conv(tmp)
        out = nn.MaxPool1d(tmp.shape[-1])(tmp)
        out = out.view(out.shape[1])

        return self.activation(self.lin(out))
        #return self.lin(out)

In [42]:
classifier = Conv1D_classifier(vocab_size=vocab_size,embedding_dim=40,out_dim=1,feat_size=20)
#loss_function = nn.MultiLabelSoftMarginLoss()
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.1)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(5):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        #loss = loss_function(probs[None,:], labl[index][None,:])
        loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        if pred.item() == labl[index].item():
          acc += 1
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

0 2000 / 10000  :  0.7392627588208167
0 4000 / 10000  :  0.6801077710833958
0 6000 / 10000  :  0.6402043162792758
0 8000 / 10000  :  0.5972731454713089
0 10000 / 10000  :  0.5643319214733974
0 tensor(0.5643) 0.696
1 2000 / 10000  :  0.2320008270736931
1 4000 / 10000  :  0.2189353338090382
1 6000 / 10000  :  0.2168148426756022
1 8000 / 10000  :  0.20985980486330322
1 10000 / 10000  :  0.20690059728629112
1 tensor(0.2069) 0.924
2 2000 / 10000  :  0.05923287495188024
2 4000 / 10000  :  0.0530112615434132
2 6000 / 10000  :  0.049889978000890775
2 8000 / 10000  :  0.04976562011657707
2 10000 / 10000  :  0.04840059811645227
2 tensor(0.0484) 0.99
3 2000 / 10000  :  0.012725665009934167
3 4000 / 10000  :  0.01350956769191077
3 6000 / 10000  :  0.013328554789649186
3 8000 / 10000  :  0.012440301490017795
3 10000 / 10000  :  0.011854746113039038
3 tensor(0.0119) 0.999
4 2000 / 10000  :  0.005497547911071014
4 4000 / 10000  :  0.005176964864995546
4 6000 / 10000  :  0.0052628110976870165
4 8000 /

KeyboardInterrupt: ignored

In [43]:
classifier.eval()

M = np.zeros((2,2))
for i in range(10000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i])
  M[real,pred] += 1
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  if len(txtidx[i]) > 2:
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(labl[i])
    M[real,pred] += 1
print("val : ", M)

train :  [[5015.    0.]
 [   0. 4985.]]
val :  [[4139.  828.]
 [ 877. 4111.]]


Une question reste : quelle loss utiliser pour le multiLabel ?

In [46]:
classifier = Conv1D_classifier(vocab_size=vocab_size,embedding_dim=40,out_dim=6,feat_size=20)
loss_function = nn.MultiLabelSoftMarginLoss()
#loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.1)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(5):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    
    for index in randomidx:
        x = txtidx[index].cuda()
        probs = classifier(x).cuda()
        loss = loss_function(probs[None,:], labl[index][None,:])
        #loss = loss_function(probs[0],labl[index].cuda())
        loss.backward()
        i +=1
        if i%batch_size == 0:
          optimizer.step()
          classifier.zero_grad()
        total_loss += loss.data
        pred = probs>0.5
        
        if i%2000 == 1999:
          print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
    print(epoch, total_loss[0]/len(randomidx))

0 2000 / 10000  :  0.6934789416192472
0 4000 / 10000  :  0.6926367211920168
0 6000 / 10000  :  0.6919599718442657
0 8000 / 10000  :  0.6913569274284286
0 10000 / 10000  :  0.6909417504250425
0 tensor(0.6909)
1 2000 / 10000  :  0.6801283942752626
1 4000 / 10000  :  0.6793724798387096
1 6000 / 10000  :  0.6798663221213952
1 8000 / 10000  :  0.6798414499078009
1 10000 / 10000  :  0.6794801550467546
1 tensor(0.6795)
2 2000 / 10000  :  0.6692844102715421
2 4000 / 10000  :  0.6677710687437485
2 6000 / 10000  :  0.6684573161282089
2 8000 / 10000  :  0.6685946827337792
2 10000 / 10000  :  0.6685227604791729
2 tensor(0.6685)
3 2000 / 10000  :  0.662834652189376
3 4000 / 10000  :  0.6631817866576019
3 6000 / 10000  :  0.6626381496525463
3 8000 / 10000  :  0.6628893186843668
3 10000 / 10000  :  0.6627567053580358
3 tensor(0.6628)
4 2000 / 10000  :  0.6600711244294022
4 4000 / 10000  :  0.6600079585326019
4 6000 / 10000  :  0.6598289249380105
4 8000 / 10000  :  0.6592019681171084
4 10000 / 10000  

In [50]:
classifier.eval()

j = 2
M = np.zeros((2,2))
for i in range(10000):
  probs = classifier(txtidx[i].cuda()).cuda()
  if probs[j] > 0.5:
    pred = 1
  else:
    pred = 0
  real = int(labl[i,j])
  M[real,pred] += 1
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  if len(txtidx[i]) > 2:
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs[j] > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(labl[i,j])
    M[real,pred] += 1
print("val : ", M)


train :  [[7293.   61.]
 [1409. 1237.]]
val :  [[7147.  252.]
 [1823.  733.]]


## Amelioration (paper 2014)

In [0]:
class Conv1D_classifier_2(nn.Module):

    def __init__(self, vocab_size, embedding_dim,out_dim, feat_size=5):
        super(Conv1D_classifier_2, self).__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(
                num_embeddings=vocab_size,
                embedding_dim=embedding_dim)
        
        
        self.conv1 = nn.Conv1d(embedding_dim,feat_size,3,padding=1)
        self.conv2 = nn.Conv1d(embedding_dim,feat_size,5,padding=2)
        self.conv3 = nn.Conv1d(embedding_dim,feat_size,7,padding=3)
        
        self.lin = nn.Linear(feat_size*3,out_dim)
        self.dropout = nn.Dropout(0.3)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        embs = self.embeddings(inputs)
        tmp=embs.view(1,self.embedding_dim,-1)
        outconv = []
        outconv.append(self.conv1(tmp))
        outconv.append(self.conv2(tmp))
        outconv.append(self.conv3(tmp))
        tmp = torch.cat(outconv, 1)
        out = nn.MaxPool1d(tmp.shape[-1])(tmp)
        out = out.view(out.shape[1])
        out = self.dropout(out)
        return self.activation(self.lin(out))
        #return self.lin(out)

In [84]:
classifier = Conv1D_classifier_2(vocab_size=vocab_size,embedding_dim=35,out_dim=1).cuda()

#loss_function = nn.MultiLabelSoftMarginLoss()
loss_function = th.nn.BCELoss()
classifier.cuda()
optimizer = torch.optim.Adagrad(classifier.parameters(),lr=0.08)
total = int(len(txtidx)*1/2)
randomidx = list(range(total))
preds = torch.zeros((total,6))
classifier.train()
batch_size = 1
for epoch in range(10):
    total_loss = torch.Tensor([0])
    np.random.shuffle(randomidx)
    classifier.zero_grad()
    i = 0
    acc = 0
    for index in randomidx:
        try:
          x = txtidx[index].cuda()
          probs = classifier(x).cuda()
          #loss = loss_function(probs[None,:], labl[index][None,:])
          loss = loss_function(probs[0],labl[index].cuda())
          loss.backward()
          i +=1
          if i%batch_size == 0:
            optimizer.step()
            classifier.zero_grad()
          total_loss += loss.data
          pred = probs>0.5
          if pred.item() == labl[index].item():
            acc += 1
          if i%2000 == 1999:
            print(epoch,i+1,'/',len(randomidx),' : ',total_loss[0].item()/i)
        except RuntimeError:
          ()
    print(epoch, total_loss[0]/len(randomidx),round(acc/i,3))

0 2000 / 10000  :  0.7386058752032266
0 4000 / 10000  :  0.7170285613395536
0 6000 / 10000  :  0.7034098782088681
0 8000 / 10000  :  0.691831168837511
0 10000 / 10000  :  0.6764208647427242
0 tensor(0.6764) 0.58
1 2000 / 10000  :  0.523586072821567
1 4000 / 10000  :  0.4967209514781039
1 6000 / 10000  :  0.49069205219099016
1 8000 / 10000  :  0.4763310562746125
1 10000 / 10000  :  0.4686456926942694
1 tensor(0.4686) 0.774
2 2000 / 10000  :  0.3219040550548712
2 4000 / 10000  :  0.3220206907195549
2 6000 / 10000  :  0.32115954465718244
2 8000 / 10000  :  0.31621796352473747
2 10000 / 10000  :  0.3110673127859661
2 tensor(0.3110) 0.866
3 2000 / 10000  :  0.2463763925419741
3 4000 / 10000  :  0.23683076836103556
3 6000 / 10000  :  0.23198619447642316
3 8000 / 10000  :  0.22488655503324634
3 10000 / 10000  :  0.2207744700251275
3 tensor(0.2208) 0.912
4 2000 / 10000  :  0.14807403701850924
4 4000 / 10000  :  0.15262464297715053
4 6000 / 10000  :  0.1494755249735633
4 8000 / 10000  :  0.1511

In [85]:
classifier.eval()

M = np.zeros((2,2))
for i in range(10000):
  try:
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(labl[i])
    M[real,pred] += 1
  except RuntimeError:
    ()
print("train : ", M)


M = np.zeros((2,2))
for i in range(10000,20000):
  try:
    probs = classifier(txtidx[i].cuda()).cuda()
    if probs > 0.5:
      pred = 1
    else:
      pred = 0
    real = int(labl[i])
    M[real,pred] += 1
  except RuntimeError:
    ()
print("val : ", M)

train :  [[5.018e+03 5.000e+00]
 [1.000e+00 4.975e+03]]
val :  [[4170.  806.]
 [ 818. 4205.]]
